In [32]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"



In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator




path  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/Buildserver/" 

names = ['Operation','TimeStamp','Process_Name','ThreadID','IrpPtr','ByteOffset','IOSize','ThreadID','ElapsedTime','DiskNum','IrpFlags','DiskSvcTime','I/O Pri','VolSnap','FileObject','FileName','IO_Pri']



all_files = glob.glob(os.path.join(path, "data-output-buildserver-7_total.csv"))
print(all_files)




['/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/Buildserver/data-output-buildserver-7_total.csv']


In [4]:
df = pd.read_csv(all_files[0],engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False) 
names = ['Operation','TimeStamp','Process_Name','ThreadID','IrpPtr','ByteOffset','IOSize','ThreadID','ElapsedTime','DiskNum','IrpFlags','DiskSvcTime','I/O Pri','VolSnap','FileObject','FileName','IO_Pri']
df.columns = names
print (len(df))

714289


In [5]:
#Sorting df by TimeStamp

df = df.sort_values(by=['TimeStamp'])
df.reset_index(inplace=True, drop=True)
print(df.dtypes)
print(len(df))

Operation        object
TimeStamp        object
Process_Name     object
ThreadID         object
IrpPtr           object
ByteOffset       object
IOSize           object
ThreadID         object
ElapsedTime      object
DiskNum          object
IrpFlags         object
DiskSvcTime      object
I/O Pri         float64
VolSnap          object
FileObject       object
FileName         object
IO_Pri           object
dtype: object
714289


In [6]:
addresses = df['ByteOffset'].tolist()
size = df['IOSize'].tolist()
addresses_dec = []
size_dec = []
count = 0

while (count < len(addresses)):
    if "Offset" in addresses[count]:
        count = count +1
        continue
    dec = addresses[count]
    dec_size = addresses[count]
    tmp = int(dec, 16)
    tmp_size = int(dec_size,16)
    addresses_dec.append(tmp)
    size_dec.append(tmp_size)
    count = count +1
    
print(len(addresses_dec))
print(len(size_dec))




714151
714151


In [7]:
from collections import Counter
print(len(Counter(df['IOSize'])))
print(len(Counter(df['ByteOffset'])))

334
396736


In [38]:
count  = 0
size_dec_norm = []
min_size = min(size_dec)
max_size = max(size_dec)
while (count < len(size_dec) ):
    tmp = (float(size_dec[count])- min_size)/(max_size - min_size)
    count  = count +1
    size_dec_norm.append(tmp)

In [39]:
len(size_dec_norm)

714151

In [40]:
df_a = pd.DataFrame(columns = ['ByteOffset', 'IOSize']) 

df_a['ByteOffset'] = addresses_dec
df_a['Size'] = size_dec_norm

df = df_a

In [41]:
df.head(5)

,ByteOffset,IOSize,Size
0,255545344,NaN,0.000218
1,33124147200,NaN,0.028233
2,54056763392,NaN,0.046075
3,15570624512,NaN,0.013272
4,732401823744,NaN,0.624265


In [42]:
df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
df = df.drop(df.index[-1])

In [43]:
df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)

a = df['ByteOffset_Delta'].unique().tolist()
operation_id_map = {}
for i,id in enumerate(a): operation_id_map[id] = i 
df['ByteOffset_Delta_class'] = df['ByteOffset_Delta'].map(lambda x: operation_id_map[x])


In [44]:
# a = df['IOSize'].unique().tolist()
# size_id_map = {}
# for i,id in enumerate(a): size_id_map[id] = i 
# df['Size_class'] = df['IOSize'].map(lambda x: size_id_map[x])

In [45]:
from collections import Counter
x = Counter(df['ByteOffset_Delta_class'])
vals = {}
vals =  x.most_common(1000)
bo_list = []

for x in vals:
    bo_list.append(x[0])
        
count = 0
label_list = []

while (count < len(df)):
    x = df['ByteOffset_Delta_class'].iloc[count]
    if x in bo_list:
        label_list.append(x)
    else:
        label_list.append(999999)
    count= count + 1
    
ByteOffset_Delta_class_backup  = df['ByteOffset_Delta_class'] 
df['ByteOffset_Delta_class']  = label_list
print(len(Counter(df['ByteOffset_Delta_class'])))
    
a = df['ByteOffset_Delta_class'].unique().tolist()
bo_map = {}
for i,id in enumerate(a): bo_map[id] = i 
df['ByteOffset_Delta_Class_1001'] = df['ByteOffset_Delta_class'].map(lambda x: bo_map[x])
    
label_list = df['ByteOffset_Delta_Class_1001'] 
    
df['ByteOffset_Delta_Class_1001']  = label_list
    

1001


In [91]:
(Counter(df['ByteOffset_Delta_Class_1001']))

Counter({0: 584420,
         1: 295,
         2: 103,
         3: 26,
         4: 110,
         5: 111,
         6: 89,
         7: 91,
         8: 100,
         9: 100,
         10: 90,
         11: 335,
         12: 29173,
         13: 2779,
         14: 523,
         15: 2035,
         16: 404,
         17: 942,
         18: 32,
         19: 110,
         20: 110,
         21: 99,
         22: 97,
         23: 93,
         24: 2155,
         25: 1033,
         26: 364,
         27: 5856,
         28: 718,
         29: 1509,
         30: 1760,
         31: 26,
         32: 346,
         33: 402,
         34: 19,
         35: 572,
         36: 95,
         37: 16,
         38: 50,
         39: 81,
         40: 464,
         41: 240,
         42: 125,
         43: 80,
         44: 203,
         45: 12,
         46: 16,
         47: 67,
         48: 19,
         49: 21867,
         50: 13,
         51: 33,
         52: 19,
         53: 70,
         54: 1089,
         55: 273,
         5

In [47]:
df.head(5)

,ByteOffset,IOSize,Size,ByteOffset_Delta,ByteOffset_Delta_class,ByteOffset_Delta_Class_1001
0,255545344,NaN,0.000218,-3.286860e+10,999999,0
1,33124147200,NaN,0.028233,-2.093262e+10,999999,0
2,54056763392,NaN,0.046075,3.848614e+10,999999,0
3,15570624512,NaN,0.013272,-7.168312e+11,999999,0
4,732401823744,NaN,0.624265,-1.901793e+09,999999,0


In [48]:
df.dtypes

ByteOffset                       int64
IOSize                          object
Size                           float64
ByteOffset_Delta               float64
ByteOffset_Delta_class           int64
ByteOffset_Delta_Class_1001      int64
dtype: object

In [49]:
from collections import Counter
print(len(Counter(df['Size'])))

396734


In [50]:
len(Counter(df['Size']))

396734

In [109]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size'].tolist()
size_test = df[training_pt_1+1:]['Size'].tolist()


In [52]:
lba_train= np.array(lba_train).reshape(-1,1)
lba_test= np.array(lba_test).reshape(-1,1)
size_train= np.array(size_train).reshape(-1,1)
size_test= np.array(size_test).reshape(-1,1)

In [53]:
# ts_train = np.arange(1,101, dtype='float64').reshape(-1,1)  
# ts_test = np.arange(21,41, dtype='float64').reshape(-1,1)
# ts_all = np.append(ts_train, ts_test).reshape(-1,1)

In [54]:
def create_dataset2(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - 2 * window_size):
        a = dataset[i:(i + window_size), 0]
        #print(a)
        dataX.append(a)
        b = dataset[(i + window_size):(i + 2* window_size), 0]
        #print(b)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

lstm_num_timesteps = 32
    
X_train_lba, y_train_lba = create_dataset2(lba_train, lstm_num_timesteps)
X_test_lba, y_test_lba = create_dataset2(lba_test, lstm_num_timesteps)
X_train_size, y_train_size = create_dataset2(size_train, lstm_num_timesteps)
X_test_size, y_test_size = create_dataset2(size_test, lstm_num_timesteps)

In [55]:
X_train_lba.shape, y_train_lba.shape, X_test_lba.shape,  y_test_lba.shape

((535548, 32), (535548, 32), (178473, 32), (178473, 32))

In [56]:
X_train_size.shape, y_train_size.shape, X_test_size.shape,  y_test_size.shape

((535548, 32), (535548, 32), (178473, 32), (178473, 32))

In [57]:
lstm_num_features = 1
lstm_predict_sequences = True
lstm_num_predictions = 32


# X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
# X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))
    

y_train_lba = np.reshape(y_train_lba, (y_train_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_test_lba = np.reshape(y_test_lba, (y_test_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_train_size = np.reshape(y_train_size, (y_train_size.shape[0], lstm_num_predictions, lstm_num_features))
y_test_size = np.reshape(y_test_size, (y_test_size.shape[0], lstm_num_predictions, lstm_num_features))                        


In [58]:
y_train_lba.shape

(535548, 32, 1)

In [61]:
# Two classification outputs
import keras

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf






from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate , Dot
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, Reshape




# no_docs = len(y_train_lba)
maxlen= 32



# # define two sets of inputs
# inputA = Input(shape=(32,))
# inputB = Input(shape=(32,))
# # inputA = Sequential()
# # inputB = Sequential()
vocabulary_1 = len(Counter(df['ByteOffset_Delta_Class_1001']))
vocabulary_2 = 50

hidden_size = 500

# input=Input(shape=(no_docs,maxlen),dtype='float64')
inputA=Input(shape=(maxlen,),dtype='float64')  
inputB=Input(shape=(maxlen,),dtype='float64') 


# the first branch operates on the first input
x = Embedding(input_dim=vocabulary_1,output_dim=hidden_size,input_length=maxlen)(inputA)
x = Model(inputs=inputA, outputs=x)

# # the second branch opreates on the second input
y = Embedding(input_dim=vocabulary_2,output_dim=hidden_size,input_length=maxlen)(inputB)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = keras.layers.concatenate([x.output, y.output])

lstm1 = LSTM(hidden_size,return_sequences=True)(combined)
lstm2 = LSTM(hidden_size, return_sequences=True)(lstm1)

# create classification output
offset = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_1, activation='softmax'), name='offset')(lstm2)
iosize = keras.layers.wrappers.TimeDistributed(Dense(units=1, activation='sigmoid'), name='iosize')(lstm2)




model =Model([inputA,inputB],[offset,iosize]) # combining all into a Keras model

model.compile(optimizer='rmsprop',
              loss={'offset': 'sparse_categorical_crossentropy', 'iosize': 'mse'},
              loss_weights={'offset': 2., 'iosize': 1.5},
              metrics={ 'offset': 'categorical_accuracy', 'iosize': 'mae'})
model.summary()


Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 32, 500)      500500      input_9[0][0]                    
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 32, 500)      25000       input_10[0][0]                   
___________________________________________________________________________________________

In [64]:
import time
num_epochs = 1000
batch_size = 32


monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights-1.hdf5", verbose=1, save_best_only=True) # save best model


print('Train...')
start_time = time.time()

model.fit([X_train_lba,X_train_size],[y_train_lba,y_train_size],
          verbose=1,epochs=1000,callbacks=[monitor,checkpointer])

model.load_weights('best_weights-1.hdf5') # load weights from best model
print('Done ...!')

#model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)


Train...
Epoch 1/1000
535548/535548 [==============================] - 1448s 3ms/step - loss: 1.7175 - offset_loss: 0.8028 - iosize_loss: 0.0746 - offset_categorical_accuracy: 0.8977 - iosize_mae: 0.1974
Epoch 2/1000
535548/535548 [==============================] - 1466s 3ms/step - loss: 1.5400 - offset_loss: 0.7153 - iosize_loss: 0.0730 - offset_categorical_accuracy: 0.8858 - iosize_mae: 0.1953
Epoch 3/1000
535548/535548 [==============================] - 1451s 3ms/step - loss: 1.4668 - offset_loss: 0.6793 - iosize_loss: 0.0721 - offset_categorical_accuracy: 0.8803 - iosize_mae: 0.1934
Epoch 4/1000
535548/535548 [==============================] - 1447s 3ms/step - loss: 1.4291 - offset_loss: 0.6607 - iosize_loss: 0.0718 - offset_categorical_accuracy: 0.8777 - iosize_mae: 0.1936
Epoch 5/1000
535548/535548 [==============================] - 1454s 3ms/step - loss: 1.4107 - offset_loss: 0.6516 - iosize_loss: 0.0717 - offset_categorical_accuracy: 0.8760 - iosize_mae: 0.1935
Epoch 6/1000
535

KeyboardInterrupt: 

In [66]:
print('Predicting...')
start_time = time.time()


pred1,pred2 = model.predict([X_test_lba,X_test_size],verbose = 1)

print("--- %s seconds ---" % (time.time() - start_time))

Predicting...
178473/178473 [==============================] - 200s 1ms/step
--- 200.26941204071045 seconds ---


In [72]:
len(pred1)

178473

In [100]:
pred_1 = pred1[:,0,:]

In [101]:
pred_11.shape

(178473,)

In [102]:
pred_11 = np.argmax(pred_1, axis=1)

In [111]:
len(lba_test)

178537

In [115]:
type(lba_test)

list

In [112]:
len(lba_test) 
test = lba_test[-178473:]

In [116]:
type(test)

list

In [132]:
predictions = list(pred_111)[0]

In [133]:
#tmp_lba = accuracy_score(y_test_lba, pred_111)

hit = 0
count = 0
while(count < len(predictions)):
    if (test[count] == predictions[count]):
        hit = hit +1
    count = count +1
    
print(hit/len(predictions))        

154661.0


In [134]:
print(hit/len(predictions)) 

0.8665792584872782


In [85]:
pred_11

array([0, 0, 0, ..., 0, 0, 0])

In [88]:
pred_111 = np.array(pred_11).reshape(1,-1)

1

In [98]:
len(lba_test)

178537

In [75]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
max_lba_accuracy = 0
max_lba_accuracy_pos = 0

min_mse_accuracy = 10000
min_mse_accuracy_pos = 0

for i in range(32):
    pred_1 = pred1[:,i,:]
    pred_2 = pred2[:,i,:]
    pred_1 = np.argmax(pred_1, axis=1)
    pred_2 = np.argmax(pred_2, axis=1)
    
    
    tmp_lba = accuracy_score(y_test_lba, pred_1)
    if (tmp_lba > max_lba_accuracy):
        max_lba_accuracy = tmp_lba
        max_lba_accuracy_pos = i
        
    tmp_size = criterion = mean_squared_error(y_test_size, pred_2)
    if (tmp_size < min_mse_accuracy):
        min_mse_accuracy = tmp_size
        min_mse_accuracy_pos = i
    

print("Min IO Size MSE", str(min_mse_accuracy))
print("Min IO Size MSE Position", str(min_mse_accuracy_pos))

print("Best LBA Delta Accuracy", str(max_lba_accuracy))
print("Best LBA Delta Position", str(max_lba_accuracy_pos))

ValueError: Classification metrics can't handle a mix of unknown and multiclass targets

In [ ]:
print("Done...!!")